# KAKENの研究機関マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、下のセルから ./config.ini として保存しておく。

In [57]:
# encoding: utf-8
import configparser
config = configparser.ConfigParser()
config['mariadb'] = {}

# それぞれ値を入力する
config['mariadb']['username'] = ''
config['mariadb']['password'] = ''
config['mariadb']['database'] = ''

with open('config.ini', 'w') as configfile:
    config.write(configfile)

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [58]:
# encoding: utf-8
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import glob
import sqlalchemy as sa
import sqlalchemy.ext.declarative
import pymysql
import configparser

In [59]:
tree = etree.parse('grants_masterxml_kaken/institution_master_kakenhi.xml')

institutionlist = []
for institution_table in tree.iterfind("institution_table"):
    for institution in institution_table.iterfind("institution"):
        institutionname = institution.find("name[@lang='ja']").text
        for code in institution.iterfind("code"):
            row = [
                code.get("type"),
                code.text,
                institutionname,
            ]
            institutionlist.append(row)
            
df = pd.DataFrame(institutionlist)
df.columns = ['code_type', 'code_number', 'institution_name']
df

,code_type,code_number,institution_name
0,nii,0010101,北海道大学
1,mext,10101,北海道大学
2,jsps,10101,北海道大学
3,nii,0010102,北海道教育大学
4,mext,10102,北海道教育大学
5,jsps,10102,北海道教育大学
6,nii,0010103,室蘭工業大学
7,mext,10103,室蘭工業大学
8,jsps,10103,室蘭工業大学
9,nii,0010104,小樽商科大学


重複のデータがあるかどうか

In [60]:
df.duplicated().any()

True

重複データがあった。dropしておく。

In [61]:
df = df.drop_duplicates()
df

,code_type,code_number,institution_name
0,nii,0010101,北海道大学
1,mext,10101,北海道大学
2,jsps,10101,北海道大学
3,nii,0010102,北海道教育大学
4,mext,10102,北海道教育大学
5,jsps,10102,北海道教育大学
6,nii,0010103,室蘭工業大学
7,mext,10103,室蘭工業大学
8,jsps,10103,室蘭工業大学
9,nii,0010104,小樽商科大学


テーブルの構造を記述する

In [62]:
Base = sa.ext.declarative.declarative_base()

class Institution(Base):
    __tablename__ = 'kaken_master_institution'
    id = sa.Column(sa.Integer, primary_key=True)
    code_type = sa.Column(sa.String(256))
    code_number = sa.Column(sa.Integer)
    institution_name = sa.Column(sa.String(256))

ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [63]:
config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [64]:
engine = sa.create_engine(url, echo=True)
Base.metadata.create_all(engine)
df.to_sql('kaken_master_institution', engine, if_exists='replace')

2018-04-06 10:01:21,174 INFO sqlalchemy.engine.base.Engine SHOW VARIABLES LIKE 'sql_mode'
2018-04-06 10:01:21,175 INFO sqlalchemy.engine.base.Engine {}
2018-04-06 10:01:21,177 INFO sqlalchemy.engine.base.Engine SELECT DATABASE()
2018-04-06 10:01:21,178 INFO sqlalchemy.engine.base.Engine {}
2018-04-06 10:01:21,180 INFO sqlalchemy.engine.base.Engine show collation where `Charset` = 'utf8' and `Collation` = 'utf8_bin'
2018-04-06 10:01:21,182 INFO sqlalchemy.engine.base.Engine {}
2018-04-06 10:01:21,184 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS CHAR(60)) AS anon_1
2018-04-06 10:01:21,185 INFO sqlalchemy.engine.base.Engine {}
2018-04-06 10:01:21,187 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS CHAR(60)) AS anon_1
2018-04-06 10:01:21,188 INFO sqlalchemy.engine.base.Engine {}
2018-04-06 10:01:21,189 INFO sqlalchemy.engine.base.Engine SELECT CAST('test collated returns' AS CHAR CHARACTER SET utf8) COLLATE utf8_bin AS anon_1
2018-04-06 10

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。